# 🧾 Explanation Generation for DSM-5 Classification using LLaMA + Instructor API

This notebook explores how to generate **rationale-based explanations** for multi-label classification decisions over the `ReDSM5` dataset. Using the `instructor` library and a local LLaMA model via Ollama, we produce structured outputs that include both:
- a **natural language explanation**, and
- a **chain-of-thought reasoning trace**.

The workflow includes:
- Loading the data and preparing test inputs,
- Defining structured output types using `pydantic`,
- Asynchronous, rate-limited inference using Ollama,
- Saving outputs for evaluation using metrics such as ROUGE and BERTScore.

This pipeline is part of a larger effort to build interpretable and clinically aligned NLP systems for mental health research.


## ⚙️ Environment Setup and Imports

We install the `instructor` library, which wraps OpenAI-compatible clients to enforce structured outputs based on `pydantic` schemas.

We also import standard libraries for:
- I/O (`json`, `requests`, `pandas`, etc.),
- Data splitting and preprocessing,
- Async inference with semaphores to avoid overload,
- Schema validation with `pydantic`.


In [ ]:
!pip install instructor


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import instructor
import asyncio
import nest_asyncio
import json
import requests

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from typing import List, Optional, Tuple
from pydantic import BaseModel, Field
from openai import AsyncOpenAI

import pandas as pd
import numpy as np

## 🔌 Connecting to Ollama (LLaMA) via Instructor

We connect to a locally running Ollama instance serving a LLaMA model using OpenAI-compatible endpoints. The `instructor.from_openai()` wrapper enforces output constraints defined by a `pydantic` schema.

A semaphore is used to limit concurrency and avoid overwhelming the local model server.


In [150]:
client = AsyncOpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
client = instructor.from_openai(client, mode=instructor.Mode.JSON)

sem = asyncio.Semaphore(5)

## 📊 Loading and Preparing ReDSM5 Data

We load the `ReDSM5` dataset and extract the input `text` and target `explanation` fields. Then we split the data into training and test sets using an 80/20 ratio.

The test portion is saved to disk in `.jsonl` format for reproducibility and batch processing.


In [151]:
data = pd.read_csv("data/redsm5.csv")

mlb = MultiLabelBinarizer()
explanations = data["explanation"].tolist()
texts = data["text"].tolist()

X_train, X_test, y_train, y_test = train_test_split(
    texts, explanations, test_size=0.2, random_state=42
)

with open("data/redsm5_test_explanations_inputs.jsonl", "w") as f:
    for text, explanation in zip(X_test, y_test):
        f.write(json.dumps({"text": text, "explanation": explanation}) + "\n")

## 📐 Defining Structured Explanation Schema

We use `pydantic` to define the structure of the model’s output:
- `explanation`: a plain-language rationale tied to specific text spans,
- `chain_of_thought`: a free-form reasoning trace of how the model inferred symptom presence or absence.

This structure ensures explanations are well-formed and interpretable.


In [152]:
class Explanation(BaseModel):
    explanation: str = Field(
        ...,
        description="Plain-language rationale that cites or paraphrases phrases from the text to justify why each symptom was tagged present or absent.",
    )

    chain_of_thought: str = Field(
        ..., description="The chain of thought that led to the explanation."
    )

## 🧠 Prompted Explanation Generation with Chain-of-Thought

We define an asynchronous function `explain()` that sends a structured prompt to the local Ollama instance running `gemma3:27b-it-qat`.

- The prompt uses few-shot examples pulled from the training set (`X_train`, `y_train`) to guide the model.
- A `system` role sets the context: the model acts as a mental-health assistant, providing DSM-5-based rationales.
- The output is parsed and validated against the `Explanation` schema, which ensures presence of both a human-readable rationale and a chain of thought.

This function is wrapped in a semaphore to manage concurrency and retry logic.


In [153]:
async def explain(data: str) -> Explanation:
    def _fewshot_messages(k=10):
        msgs = []
        for x, y in zip(X_train[:k], y_train[:k]):
            msgs.append({"role": "user", "content": x})
            msgs.append({"role": "assistant", "content": y})
        return msgs

    async with sem:
        try:
            result = await client.chat.completions.create(
                model="gemma3:27b-it-qat",
                response_model=Explanation,
                max_retries=5,
                messages=[
                    {
                        "role": "system",
                        "content": (
                            "You are an expert mental-health assistant. "
                            "Your job is to read a first-person text (e.g. a social-media post) and explain which DSM-5 A-criteria symptoms for Major Depressive Disorder are directly present or explicitly absent in that text."
                        ),
                    },
                    *(_fewshot_messages(k=50)),
                    {"role": "user", "content": data},
                ],
            )
            return result
        except Exception as e:
            print(f"Error during API call: {e}")
            return Explanation(
                explanation="Error during explanation.",
                chain_of_thought="Error during explanation.",
            )

## 📬 Batch Processing and Output Writing

We define `process_inputs()` to loop through a list of test inputs, call the `explain()` function on each, and store the results.

Each output contains:
- the original text,
- the ground-truth explanation (`explanation_true`),
- the predicted explanation and reasoning trace.

All results are saved in JSON Lines format to support later evaluation and inspection.


In [154]:
async def process_inputs(inputs: List[dict], output_path: Optional[str] = None) -> None:
    for input_data in inputs:
        prediction = await explain(input_data["text"])

        resp = {
            "text": input_data["text"],
            "explanation_true": input_data["explanation"],
            "explanation_predicted": prediction.explanation,
            "chain_of_thought": prediction.chain_of_thought,
        }

        if output_path:
            with open(output_path, "a") as f:
                json.dump(resp, f)
                f.write("\n")

## 🧪 Inference on the Test Set

We read the previously saved test inputs (`redsm5_test_explanations_inputs.jsonl`) and run the asynchronous generation pipeline.

Predictions are written to `redsm5_test_explanations_outputs.jsonl`. Errors or schema mismatches are printed to the console for debugging.

In [155]:
with open("data/redsm5_test_explanations_inputs.jsonl", "r") as f:
    inputs = [json.loads(line) for line in f]

nest_asyncio.apply()
await process_inputs(inputs, output_path="data/redsm5_test_explanations_outputs.jsonl")

Error during API call: 1 validation error for Explanation
chain_of_thought
  Field required [type=missing, input_value={'explanation': "This is ...nation': 'This is ...}"}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
Error during API call: 1 validation error for Explanation
chain_of_thought
  Field required [type=missing, input_value={'explanation': "This nar...'mental health crisis']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
Error during API call: 1 validation error for Explanation
chain_of_thought
  Field required [type=missing, input_value={'explanation': "The text... deception', 'despair']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
Error during API call: 1 validation error for Explanation
  Invalid JSON: EOF while parsing an object at line 22 column 2 [type=json_invalid, input_value='{\n  "explanation": "Thi... \n\n  \n\n  \n  

## 🔗 Embedding-Based Evaluation

We define `get_ollama_embedding()` to generate sentence embeddings for a given explanation using Ollama’s embedding endpoint (e.g., `nomic-embed-text`).

These embeddings will be used to compute cosine similarity between the ground-truth and predicted explanations, as a meaning-based evaluation metric (complementary to n-gram methods like ROUGE).

In [168]:
def get_ollama_embedding(text, model="nomic-embed-text"):
    url = "http://localhost:11434/api/embed"
    payload = {"model": model, "input": text}
    resp = requests.post(url, json=payload)
    resp.raise_for_status()
    data = resp.json()
    embedding = data.get("embeddings", [])
    if not embedding:
        print(f"[WARN] Empty embedding for text: '{text[:60]}...'")
        return np.zeros(768, dtype=np.float32)
    if isinstance(embedding[0], list):
        embedding = embedding[0]
    return np.array(embedding, dtype=np.float32)

## 📏 Semantic Similarity with Cosine Distance

We implement `compute_embedding_similarity()` to:
- Read predicted and true explanations from the JSONL file,
- Embed both using Ollama's embedding API,
- Compute cosine similarity between each pair,
- Report the average similarity as a proxy for how semantically aligned the predictions are with the human-written explanations.

This offers a flexible evaluation signal that tolerates paraphrasing and rewording.


In [173]:
def compute_embedding_similarity(jsonl_path, model="nomic-embed-text"):
    refs_pred, hyps_pred = [], []
    entries = []

    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            true = obj["explanation_true"]
            pred = obj["explanation_predicted"]
            entries.append((true, pred))
            refs_pred.append(true)
            hyps_pred.append(pred)

    gptscore_pred = []
    for true, pred in entries:
        emb_true = get_ollama_embedding(true, model)
        emb_pred = get_ollama_embedding(pred, model)

        norm_true = np.linalg.norm(emb_true)
        norm_pred = np.linalg.norm(emb_pred)

        if norm_true == 0 or norm_pred == 0:
            cos_pred = 0.0
        else:
            cos_pred = np.dot(emb_true, emb_pred) / (norm_true * norm_pred)
            cos_pred = float(np.clip(cos_pred, -1.0, 1.0))

        gptscore_pred.append(cos_pred)

    def avg(scores):
        return float(np.mean(scores)) if scores else 0.0

    print(f"Avg. embedding similarity: {avg(gptscore_pred):.4f}")

## 📈 Similarity Results

We call `compute_embedding_similarity()` to compute the average cosine similarity between ground-truth and predicted explanations using `nomic-embed-text` embeddings.

This provides a semantics-aware score that reflects how close the generated rationales are to the expert annotations, even if they differ in surface form.

> ✅ Result: A mean similarity of ~0.78 suggests the model generates mostly on-topic and clinically relevant explanations.


In [174]:
compute_embedding_similarity(
    jsonl_path="data/redsm5_test_explanations_outputs.jsonl",
    model="nomic-embed-text",
)

Avg. embedding similarity: 0.7846


## 🧪 LLM-Based Explanation Judging

To complement embedding-based metrics, we define a `JudgeScores` schema that includes:
- `accuracy`: agreement with clinical facts,
- `coverage`: how completely the explanation addresses relevant symptoms,
- `clarity`: readability and fluency.

These are rated by a second LLM (`deepseek-r1 8B`) acting as an expert evaluator using a few-shot rubric. The judge returns a numeric score (1–5) for each aspect and a brief justification.


In [221]:
class JudgeScores(BaseModel):
    accuracy: int
    coverage: int
    clarity: int
    explanation: str

## 🤖 Explanation Rating with Chain-of-Thought Rubric

The `llm_judge()` function prompts the LLM to compare a candidate explanation against the expert reference using natural language instructions and structured output validation.

A weighted average score is computed using:
- 40% for accuracy,
- 30% for coverage,
- 30% for clarity.

This simulates human-style evaluation while ensuring standardization across examples.


In [222]:
async def llm_judge(
    reference: str, candidate: str, model: str
) -> Tuple[float, JudgeScores]:
    WEIGHTS = {"accuracy": 0.40, "coverage": 0.30, "clarity": 0.30}

    system_msg = {
        "role": "system",
        "content": (
            "You are an expert evaluator of short clinical explanations.\n"
            "Rate the CANDIDATE against the REFERENCE with this rubric:\n"
            "1. Clinical Accuracy (1-5): Does the candidate broadly match the clinical statements? "
            "If there are no significant contradictions and the main points align, you may rate 4 or 5.\n"
            "2. Coverage (1-5): If the candidate mentions most of the important symptoms or absences described in the reference, and does not invent major content, you may give a high score. "
            "Minor omissions are acceptable if the main message remains the same.\n"
            "3. Clarity & Conciseness (1-5): Is the candidate easy to read and generally clear?\n\n"
            "If the candidate explanation broadly covers the same clinical content or intent as the reference, you may give a high score, even if the phrasing or minor details differ.\n"
            "Return ONLY a JSON object with keys accuracy, coverage, clarity and explanation (≤2 lines)."
        ),
    }

    user_msg = {
        "role": "user",
        "content": (
            f"REFERENCE:\n<<<{reference}>>>\n\n" f"CANDIDATE:\n<<<{candidate}>>>"
        ),
    }

    async with sem:
        try:
            result: JudgeScores = await client.chat.completions.create(
                model=model,
                response_model=JudgeScores,
                max_retries=5,
                messages=[system_msg, user_msg],
            )
        except Exception as e:
            result = JudgeScores(
                accuracy=3, coverage=3, clarity=3, explanation="Error during judging."
            )

    overall = sum(WEIGHTS[k] * (getattr(result, k) - 1) / 4 for k in WEIGHTS) * 100
    return overall, result

## 📤 Batch Evaluation of LLM-Generated Explanations

We define `evaluate_jsonl_llm_judge()` to:
- Iterate over all test examples,
- Send each reference/prediction pair to the judge LLM,
- Collect and average the evaluation scores.

This provides a final model-level score based on human-aligned qualitative criteria.


In [223]:
async def evaluate_jsonl_llm_judge(path: str, model: str) -> None:
    with open(path, encoding="utf-8") as f:
        data = [json.loads(line) for line in f]

    async def judge(ref: str, hyp: str):
        score, _ = await llm_judge(ref, hyp, model)
        return score

    tasks_pred = []
    for obj in data:
        ref = obj["explanation_true"]
        pred = obj["explanation_predicted"]
        tasks_pred.append(judge(ref, pred))

    pred_scores = await asyncio.gather(*tasks_pred)

    print(f"Avg. llm judge score: {np.mean(pred_scores):6.2f}")

## 🧾 Final Results: LLM Judge Score

We run the evaluation on all test examples and print the average judge score. A result around 61.5 (on a 0–100 scale) suggests that most generated explanations are accurate, reasonably complete, and clearly written, though there is still room for improvement.

In [224]:
asyncio.run(
    evaluate_jsonl_llm_judge(
        "data/redsm5_test_explanations_outputs.jsonl", model="deepseek-r1:8b"
    )
)

Avg. llm judge score:  61.50
